# Explore data in Database

This notebook describes how to connect to the database and query the stored measurments

In [ ]:
from datetime import datetime

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from p1reader.sinks import DBSink, DBSinkConfig
import p1reader.sinks.db_operations as op

## Connect to database and init data source

In [ ]:
config = DBSinkConfig(host="192.168.10.1", port=5432, database="premises", user="postgres", password="password")

In [ ]:
sink = DBSink(config.output_stream)

## Get meters in the DB

In [ ]:
ids = sink.query_meter_ids()
print(ids)

## Get range of data for first meter

In [ ]:
start_date, end_date = sink.query_date_range(ids[0])
print(start_date)
print(end_date)

## Get number of phases

In [ ]:
phases = sink.query_n_phases(ids[0])
print(phases)

## Get Mbus devices for first meter

In [ ]:
mbus = sink.query_mbus_ids(ids[0])
print(mbus)

## Query data Electrical data

In [ ]:
measurements = ["P+", "P-", "U(L1)"]

In [ ]:
results = sink.query_sql(device_id=ids[0], start_date=datetime(2023, 12, 22), end_date=datetime(2023,12,23), measurements=measurements, sql_collection=op.ELEC)

## Convert raw results to pandas df

In [ ]:
df_elec = pd.DataFrame()
for measure, values in results.items():
    readings = [value[3] for value in values]
    indexes = [value[0] for value in values]
    df_temp = pd.DataFrame(index=indexes, data=readings, columns=[measure])
    df_elec = pd.concat([df_elec, df_temp], axis=1)
df_elec.head()

## Query MBus data

In [ ]:
results = sink.query_sql(device_id=ids[0], start_date=datetime(2023, 12, 22), end_date=datetime(2023,12,23), measurements=mbus, sql_collection=op.MBUS)

In [ ]:
df_mbus = pd.DataFrame()
for measure, values in results.items():
    readings = [value[4] for value in values]
    indexes = [value[0] for value in values]
    df_temp = pd.DataFrame(index=indexes, data=readings, columns=[measure])
    df_mbus = pd.concat([df_mbus, df_temp], axis=1)
df_mbus.head()

In [ ]:
results = sink.query_sql(device_id=ids[0], start_date=start_date, end_date=end_date, sql_collection=op.PEAK)

In [ ]:
df_peak = pd.DataFrame()
for measure, values in results.items():
    readings = [value[2] for value in values]
    indexes = [value[0] for value in values]
    df_temp = pd.DataFrame(index=indexes, data=readings, columns=[measure])
    df_peak = pd.concat([df_peak, df_temp], axis=1)
df_peak.head()

In [ ]:
results = sink.query_sql(device_id=ids[0], start_date=start_date, end_date=end_date, sql_collection=op.PEAK_HISTORY)

In [ ]:
df_peak_history = pd.DataFrame()
for measure, values in results.items():
    readings = [value[3] for value in values]
    indexes = [value[0] for value in values]
    df_temp = pd.DataFrame(index=indexes, data=readings, columns=[measure])
    df_peak_history = pd.concat([df_peak_history, df_temp], axis=1)
df_peak_history.head()

## Plot results

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(df_elec["P+"])
plt.xlabel("Date")
plt.ylabel("Power [kW]")
plt.tight_layout()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(df_elec["U(L1)"])
plt.xlabel("Date")
plt.ylabel("Voltage [V]")
plt.tight_layout()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(df_mbus)
plt.xlabel("Date")
plt.ylabel("Measurement")
plt.tight_layout()

In [ ]:
plt.figure(figsize=(10,6))
plt.bar(x=df_peak.index, height=df_peak["Default"].values)
plt.xlabel("Date")
plt.ylabel("Power [kW]")
plt.tight_layout()

In [ ]:
plt.figure(figsize=(10,6))
plt.bar(x=df_peak_history.index, height=df_peak_history["Default"].values)
plt.xlabel("Date")
plt.ylabel("Power [kW]")
plt.tight_layout()